## 成品构造训练数据，采用本地Qwen3-32B 模型用LangChain

## 定义变量

In [1]:
from openai import OpenAI

# 定义变量
model="Qwen3-32B"
base_url="http://192.168.121.59:9008/v1"
api_key="none"

## 使用 LangChain 构造训练数据

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(base_url=base_url,
                  api_key=api_key,
                  model=model,
                  temperature=1,
                  max_tokens=4095)

### 数据增强：构造多样化的提问方式

In [3]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

## 自动化批量生成训练数据流水线

In [4]:
## 去掉双引号
def remove_all_quotes(s):
    if isinstance(s, str):
        return s.replace('"', '')
    return s  # 非字符串类型原样返回

In [5]:
## 按行分析LLM返回内容
def parse_content_summary_line_by_line(text):
    content = None
    summary_lines = []
    in_summary = False
    in_quote = False

    # 按行分割
    lines = text.splitlines()

    for line in lines:
        line = line.strip()

        # 跳过空行和 think 标签
        if not line or line.startswith('<think>') or line.endswith('</think>'):
            continue

        # 处理 content
        if line.startswith('content:') and not content:
            # 提取 content: 后面双引号中的内容
            start = line.find('"')
            if start != -1:
                end = line.find('"', start + 1)
                if end != -1:
                    content = line[start + 1:end]
                else:
                    # 只有开头引号，没有闭合 → 取到行尾（去掉引号）
                    content = line[start + 1:]

        # 处理 summary
        if line.startswith('summary:') and not in_summary:
            in_summary = True
            # 找到 summary: 后的第一个 "
            start = line.find('"')
            if start != -1:
                # 从 " 之后的内容开始收集
                part = line[start + 1:]
                if part:
                    summary_lines.append(part)
                in_quote = True
            else:
                # 没有引号，直接从后面开始
                after = line[8:].strip()  # len('summary:') == 8
                if after:
                    summary_lines.append(after)
                in_quote = False
        elif in_summary:
            # 如果 summary 已开始，继续收集所有非空行，直到我们认为结束
            # 判断是否到了外部的 '''（常见于多行字符串结尾）
            if "'''" in line or '"""' in line or line.startswith('</think>'):
                break  # 结束
            summary_lines.append(line)

    summary = '\n'.join(summary_lines) if summary_lines else None

    return  remove_all_quotes(content), remove_all_quotes(summary)


In [6]:
def gen_data(raw_content):
    """
    使用LLM调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: LLM模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""/no_think 
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [7]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = parse_content_summary_line_by_line(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)

In [8]:
# 执行主函数
main()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19968\341638561.py:42: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = chat(messages)


Content: 蒙卦
Summary: 在周易中，蒙卦象征着启蒙与迷茫的双重状态，其卦象为下坎（水）上艮（山），意味着山下泉水初涌，象征事物初始阶段的混沌与未知。《易经》中，蒙卦代表智慧尚未开启、内心蒙昧的状态，强调教育与引导的重要性。
蒙卦卦辞中提到：“匪我求童蒙，童蒙求我”，揭示了启蒙的本质：并非主动去求教于蒙昧者，而是蒙昧者主动寻求开导。此卦强调敬畏与真诚，若占卜时态度不敬，反会被忽视。因此，《象辞》以“山下出泉”象征启蒙的开始，主张君子应效法山泉之坚定与果敢，以行动培养德行。
传统解释中，蒙卦为离宫四世卦，有回环往复之意，象征事物在初期多疑虑、困惑，但只要时机得当，行动果断，便有启蒙通达之可能。邵雍称其为“智慧未开，蒙昧闭塞”，若能顺从贤师良友，则可开智亨通。傅佩荣则强调蓄德用世，主张在德行积累之后方能有所作为。
从运势上看，蒙卦象征初时迷惑，需耐心等待良机，多听他人意见。在事业上，混乱无序中需有勇有决，以坚定的行动扭转局面，并通过教育提升自我。经商方面应谨慎行事，树立信誉。婚恋中应注重品德考察，避免物质诱惑。总体而言，蒙卦虽属启蒙之象，然亦含凶机，若能顺时顺师，必有所成。
Content: 屯卦
Summary: 在《周易》中，屯卦象征万物初生之艰难，是坎（水）上震（雷）下所组成的卦象，代表雷雨交加、环境险恶的状态。其卦名“屯”意为“难”，象征着万事万物在萌发之初，面临着重重阻碍与挑战，必须刚毅果敢、持之以恒，方能突破困境，逐步通达。
屯卦的卦辞为：“元，亨，利，贞。勿用，有攸往，利建侯。”意思是初始吉利，通达顺利，适宜占卜，但不宜贸然行动，应守而不进，利于建立诸侯之功。《象辞》曰：“云雷屯，君子以经纶。”说明君子应效法云雷之象，以温和的恩泽与果断的行动治理国家，化险为夷。
哲学层面，屯卦蕴含“万物始生、先劳后逸”的深意。邵雍解其为“万物始生，开始困难；先劳后逸，苦尽甘来”，表明艰难是成功的前提，唯有经过努力与坚持，才能迎来光明的未来。得此卦者，宜守不宜进，须耐心应对困境，排除障碍，才能最终通达。
《断易天机》认为屯卦为“动而逢险”，需刚毅果敢方可吉利。而傅佩荣则从时运、财运、家宅、身体等多方面指出其象征意义，强调“宜守不宜进”。
整体而言，屯卦是困境与潜力并存的象征，提醒人们在逆境中保持坚韧，积极进取，在等待时机的同时不断积蓄力量，终将苦尽甘来，迎来新